# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Цель исследования
На основе входных данных выявить зависимости между семейным положением, количеством детей и фактом погашения кредита в срок для построения модели кредитного скоринга.

## Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Изучение данных

In [1]:
#импортируем библиотеки
import pandas as pd
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter

df = pd.read_csv('/datasets/data.csv')
df.info()
#проверка уникальных значений в столбцах таблицы циклом:
for col in df.columns:
    print(df[col].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413    

#### Комментарий

1) В данных по стажу работы и общий доход есть пропуски.Возможными причинами пропусков могут быть отсутствие данных в централизованных источниках (например, если человек не работал официально) из которых могли браться данные и/или нежелание указывать клиентами при заполнении анкет, если данные брались на основании, указанной клиентами информации.

2) В столбце стаж работы в днях и дети есть отрицательные значения. Данные по трудовому стажу не являются необходимыми для ответа на поставленные вопросы в задаче.
Вероятно, технологические ошибки. Их можно заменить на положительные для корректных расчетов характерных значений.

**Вывод**

В выборке 21 525 человек из них больше всего по критериям:
- Семейное положение: Женаты / замужем - 12 380 человек > 50%
- Пол: Мужчины - 14 236 человек > 50%
- Тип дохода: Сотрудники 11 119 человек > 50%
- Дети: Бездетные - 14 149 человек > 50%

## Предобработка данных

#### Комментарий

Тип данных в столбцах стаж работы и общий доход c пропусками - float, переменные - количественные --> логика работы с количественными переменными, т.е. заполнение пропусками характерными значениями (среднее или медиана). 

In [2]:
#перевод отрицательных значений трудового стажа и детей в положительные
df['days_employed'] = df['days_employed'].abs()
df['children'] = df['children'].abs()

#просчитаем средние и медианные значения в столбцах с пропущенным стажем в днях
avr_days_employed_before = df['days_employed'].mean()
median_days_employed_before = df['days_employed'].median()
avr_total_income_before = df['total_income'].mean()
median_total_income_before = df['total_income'].median()
print('Средний стаж в днях:', avr_days_employed_before)
print('Медианный стаж в днях:', median_days_employed_before)
print('Средний ежемесячный доход:', avr_total_income_before)
print('Медианный ежемесячный доход:', median_total_income_before)

#заменим пропущенный стаж в днях на медианные значения
df['days_employed'] = df['days_employed'].fillna(value = median_days_employed_before)


#заменим пропущенные значения по доходу на медиану по соответствующему типу доходов
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))
median_total_income_after = df['total_income'].median()
print()
#расчитаем медианный доход по типу доходов в порядке убывания и выведем на экран
income_type_median = df.groupby('income_type')['total_income'].median()
income_type_median_sorted = income_type_median.sort_values(ascending=False)
print('Медианный по типам дохода:', income_type_median_sorted)
print()
print('Медианный ежемесячный доход после замены пропусков:', median_total_income_after)
df.info()

Средний стаж в днях: 66914.72890682236
Медианный стаж в днях: 2194.220566878695
Средний ежемесячный доход: 167422.30220817294
Медианный ежемесячный доход: 145017.93753253992

Медианный по типам дохода: income_type
предприниматель    499163.144947
компаньон          172357.950966
госслужащий        150447.935283
сотрудник          142594.396847
безработный        131339.751676
пенсионер          118514.486412
студент             98201.625314
в декрете           53829.130729
Name: total_income, dtype: float64

Медианный ежемесячный доход после замены пропусков: 142594.39684740017
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender         

#### Комментарий

1) Обнаружены значительные различия между средним и медианным стажем по стажу работы. 
Причина - наличие выбросов (например,стаж у одного из клиента 401 755 дней = 1100 лет). Просто удалить выбросы нельзя, т.к. в нем присутствует информация о доходе, необходимая для ответа на один из вопросов задачи, поэтому можно оставить как есть для экономии времени. 

2) По доходу принято решение для замены пропусков использовать медиану, т.к. возможно наличие выбросов, искажающих среднее значение. Кроме того выявлены различия в медианном доходе по разным типам дохода, поэтому пропуски заменены на медианы по соответствующим группам, а не на медиану по всей выборке.

**Вывод**

Видно, что доходы значительно различаются по типам: у предпринимателей наибольший медианный доход, у клиентов в декрете - наименьший. 

### Замена типа данных

In [3]:
#Замена вещественного типа данных на целочисленные методом astype(), т.к. метод to_numeric переводит данные в тип float.
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Для столбцов с вещественным типом float64 произведена замена типа данных на целочисленный int.

### Обработка дубликатов

In [4]:

#приведение названий ступеней образования к нижнему регистру 
df['education'] = df['education'].str.lower()
print('Уникальные значения по образованию:')
print(df['education'].value_counts())
print()

#подсчет полных дубликатов
print('Полных дубликатов в таблице:', df.duplicated().sum())
#удаление полных дубликатов
df = df.drop_duplicates().reset_index(drop= True)
df.shape

Уникальные значения по образованию:
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Полных дубликатов в таблице: 71


(21454, 12)

#### Комментарий

1) Проверены уникальные значения в основных категориальных переменных, где возможны дубликаты в текстовых данных
2) Выявлены дубликаты по признаку "образование" из-за разного регистра. Регистр для всех заменен на нижний.
3) Проверены и удалены полные дубликаты методом drop_duplicates()

**Вывод**

Теперь у нас есть корректная информация по образованию. Наибольшее кол-во заемщиков в выборке имеют среднее образование.

### Лемматизация

In [5]:
#формируем список уникальных значений целей кредита
purpose_unique = df['purpose'].unique()
#преобразование в единый текст-строку
purpose_unique_str = ' '.join(purpose_unique)
#подсчет частоты лемм в столбце purpose 
lemmas = m.lemmatize(purpose_unique_str)
lemmas_counted = Counter(lemmas)
#вывод
print(lemmas_counted)

#формируем столбец с леммами для каждой строчки данных
def lemmatization(row):
        lemmas = m.lemmatize(row)
        return lemmas
df['lemmas'] = df['purpose'].apply(lemmatization)
print(df['purpose'].head(10))
print(df['lemmas'].head(10))
    

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})
0                 покупка жилья
1       приобретение автомобиля
2                 покупка жилья
3    дополнительное образование
4               сыграть свадьбу
5                 покупка жилья
6             операции с жильем
7                   образование
8         на проведение свадьбы
9       покупка жилья для семьи
Name: purpose, dtype: object
0                      [покупка,  , жилье, \n]
1            [приобретение,  , автомобиль, \n]
2                      [покупка,  , жилье, \n]
3         [дополнительный,  , образование, \n]
4      

#### Комментарий

После лемматизации выделены повторяющиеся леммы, которые могут быть сгруппированы по следующим категориям:
1. недвижимость, жилье - недвижимость
2. образование - образование
3. автомобиль - автокредит
4. свадьба - свадьба

Также был сформирован столбец с леммами на основе значений purpose для дальнейшей категоризации. 

**Вывод**

Основные целей обращений: недвижимость, образование, автокредит, свадьба

### Категоризация данных

In [6]:
#присвоение будущим категориям id
category_df = pd.DataFrame([[1, 'недвижимость'], [2,'образование'], [3,'автокредит'], [4,'свадьба']], columns=['purpose_id','purpose_common_name'])

#функция присвоение категорий
def categorization(data):
        if 'жилье' in data or 'недвижимость' in data:
            return 1
        if 'образование' in data:
            return 2
        if 'автомобиль' in data:
            return 3
        if 'свадьба' in data:
            return 4
df['purpose_id'] = df['lemmas'].apply(categorization)
#подтянем столбец с названиями категорий к значениям методом merge()
df = df.merge(category_df, on = 'purpose_id', how = 'left')
print(df['purpose_common_name'].value_counts())

недвижимость    10811
автокредит       4306
образование      4013
свадьба          2324
Name: purpose_common_name, dtype: int64


**Вывод**

Больше чем у половины клиентов в выборке цель кредита связана с недвижимостью.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
#функции замены названий столбцов в сводной таблицы и расчета доли просрочек
def column_name_change(data):
    data.set_axis(['debt_N', 'debt_Y'], axis = 'columns', inplace = True)
    return data
def ratio_calc(data):
    data['debtors_ratio'] = data['debt_Y'] / (data['debt_Y'] + data['debt_N'])
    data = data.sort_values(by = 'debtors_ratio', ascending = False)
    return data


#построение сводной таблицы
data_pivot_children_debt = df.pivot_table(index=['children'], columns='debt', values='gender', aggfunc='count')
#замена названий столбцов и расчет доли должников
column_name_change(data_pivot_children_debt)
ratio_calc(data_pivot_children_debt).style.format({'debtors_ratio': '{:.2%}'})

,debt_N,debt_Y,debtors_ratio
children,,,
20,68,8,10.53%
4,37,4,9.76%
2,1858,194,9.45%
1,4410,445,9.17%
3,303,27,8.18%
0,13028,1063,7.54%
5,9,nan,nan%


**Вывод**

В данных прослеживается зависимость - по мере уменьшения кол-ва детей доля клиентов с просрочкой возврата падает с 10,5% для клиентов с 20 детьми и 7,5%
для клиентов без детей. 
Среди клиентов с 5 детьми должников не оказалось, но их всего 9 человек в выборке, поэтому необходимо  дополнительное исследование таких клиентов в будущем для более точного подтверждения вывода.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
#построение сводной таблицы
data_pivot_family_debt = df.pivot_table(index=['family_status'], columns='debt', values='gender', aggfunc='count')
#замена названий столбцов и расчет доли должников
column_name_change(data_pivot_family_debt)
ratio_calc(data_pivot_family_debt).style.format({'debtors_ratio': '{:.2%}'})

,debt_N,debt_Y,debtors_ratio
family_status,,,
Не женат / не замужем,2536,274,9.75%
гражданский брак,3763,388,9.35%
женат / замужем,11408,931,7.55%
в разводе,1110,85,7.11%
вдовец / вдова,896,63,6.57%


**Вывод**

В данных прослеживается зависимость. 
Можно выделить три группы категорий по величине % невозврата:
1. Категории "Не женат / не замужем" и "гражданский брак" имеют наибольший % невозвратов (9,8% и 9,5% соответственно)
2. Категории "женат / замужем и "в разводе" имеют % невозвратов 7,5% и 7,1% соответственно
3. Категория "вдовец / вдова" имеют наименьший % невозвратов - 6,6%

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

#### Комментарий

Как мы выяснили ранее, данные по доходам у разных типов доходов различаются. Кроме того, в выборке >50% относятся к сотрудникам. Это значит, что для более объективной картины целесообразно сравнивать характерные значения по типам дохода с точки зрения выплаты/невыплаты кредита в срок.

In [9]:
#сортируем таблицу по возрастанию дохода
df = df.sort_values(by = 'total_income', ascending = True)
#print(df.head())

#добавление в таблицу исходных данных столбцов с разделением на квинтили с помощью метода qcut c диапазонами значений и с номерами групп


df['income_quintiles'] = pd.qcut(df['total_income'],q = 5, precision=0)
income_groups = ['1', '2', '3', '4', '5']
df['income_quintiles_label'] = pd.qcut(df['total_income'],q = 5 ,labels= income_groups)
#print(df.head(5))

#посмотрим, какие выделены группы доходов и как распределены клиенты по группам доходов
print(df['income_quintiles'].unique())
print(df['income_quintiles'].value_counts())
print(df['income_quintiles_label'].value_counts())


#построение сводной таблицы
data_pivot_income_debt = df.pivot_table(index=['income_quintiles_label'], columns='debt', values='gender', aggfunc='count')
#замена названий столбцов и расчет доли должников
column_name_change(data_pivot_income_debt)
ratio_calc(data_pivot_income_debt).sort_values(by = 'debtors_ratio', ascending = True).style.format({'debtors_ratio': '{:.2%}'})

[(20666.0, 98538.0], (98538.0, 132134.0], (132134.0, 161335.0], (161335.0, 214618.0], (214618.0, 2265604.0]]
Categories (5, interval[float64]): [(20666.0, 98538.0] < (98538.0, 132134.0] < (132134.0, 161335.0] < (161335.0, 214618.0] < (214618.0, 2265604.0]]
(214618.0, 2265604.0]    4291
(161335.0, 214618.0]     4291
(98538.0, 132134.0]      4291
(20666.0, 98538.0]       4291
(132134.0, 161335.0]     4290
Name: income_quintiles, dtype: int64
5    4291
4    4291
2    4291
1    4291
3    4290
Name: income_quintiles_label, dtype: int64


,debt_N,debt_Y,debtors_ratio
income_quintiles_label,,,
5,3991,300,6.99%
1,3947,344,8.02%
2,3930,361,8.41%
4,3930,361,8.41%
3,3915,375,8.74%


**Вывод**

<span style="color:purple">**В результате анализа видно, что уровень доходов влияет на факт невозврата.**

- Среди клиентов с высокими доходами в 5 группе (20% клиентов с диапазоном доходов 214 618 - 2 265 604 руб) наименьший % невозвратов - 7%
- Клиенты 1 группы (20 666 - 98 538) на втором месте по надежности - 8%
- Клиенты  из 2 (98 538 - 132 134 руб) и 4 (161 335 - 214 618 руб) группы имеют равный % невозврата 8,4%
- Наибольший % невозвратов в 3 группе с доходами в диапазоне  (132 134 - 161 335 руб.) - 8,7%

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
#построение сводной таблицы
data_pivot_purpose_debt = df.pivot_table(index=['purpose_common_name'], columns='debt', values='gender', aggfunc='count')
#замена названий столбцов и расчет доли должников
column_name_change(data_pivot_purpose_debt)
ratio_calc(data_pivot_purpose_debt).style.format({'debtors_ratio': '{:.2%}'})

,debt_N,debt_Y,debtors_ratio
purpose_common_name,,,
автокредит,3903,403,9.36%
образование,3643,370,9.22%
свадьба,2138,186,8.00%
недвижимость,10029,782,7.23%


**Вывод**

Самые рискованные с точки зрения невозврата цели - автокредит и образование (9% доля невозвратов)
Далее идет свадьба (8% доля невозвратов)
Меньше всего невозвратов для недвижимости (7%) - хорошая новость, т.к. доля этих клиентов в нашей статистике > 50%

## Шаг 4. Общий вывод

**В ходе исследования были выявлены следующие факты в данных:**

- По критериям пол, тип дохода, семейное положение, дети более половины наблюдений приходится на одно их значение. Это может влиять на репрезентативность результатов исследования. Возможно есть дополнительные данные для уточнения выборки.
- Существуют пропущенные значения критерия дохода и стаже. Это может быть связано с отсутствием данных в централизованных источниках, если данные брались автоматически и/или нежелание указывать клиентами при заполнении заявок. 
- В данных о кол-ве детей и стаже работы в днях есть отрицательные значения. Скорее всего технологическая ошибка при консолидации данных из разных систем.
- Данные об образовании заполнены с разными регистрами

Для решения поставленной задачи данные были обработаны: пропущенные значения заменены медианными, отрицательные переведены в положительные, изменен тип данных в столбцах стажа и дохода, регистры приведены к нижнему, дубликаты удалены, проведены категоризация и лемматизация

**В результате были обнаружены зависимости между количеством детей, семейным положением и фактом невозврата кредита в срок:**
- Зависимость между кол-вом детей и невозвратом - обратная, т.е. чем больше детей, тем больше доля должников.Самые привлекательные заемщики - бездетные (ок.8% невозврата).
- Семейное положение также влияет на факт невозврата: не состоящие в браке, либо состоящие в гражданском браке заемщики - с самой большой долей невозврата (9-10%).

**Кроме того, были исследовано влияние уровня дохода и цели кредита на факт невозврата:**
- Выявлено влияние уровня дохода на невозврат кредитов - клиенты в верхнем диапазоне доходов имеют наименьший % невозвратов (7%), далее идут клиенты из нижнего диапазона дохода(8%). Клиенты с доходами 161 335 - 214 618 имеют наибольший % невозвратов - 8,7%
- Есть влияние цели кредита - наиболее рискованные заемщики с целью автокредит и образование (9%), наименее рискованные - недвижимость(7%)
Таким образом, цель исследования выполнена.

Возможное направление для дальнейших исследований - влияние возраста, пола, образования на невозврат кредитов.

**Рекомендации:**
1. Проверить выборку, возможно, есть дополнительные данные, которые сделают ее более репрезентативной
2. Сделать поле дохода обязательным для заполнения, если они заполняются клиентами самостоятельно, для исключения пропусков.
3. Проверить как заполнены данные по кол-ву детей и стаже работы в днях, чтобы выявить причину отрицательных значений.
4. Ввести требование к заполнению поля образования только нижним регистром.